In [1]:
#Import Python libraries

import numpy as np
import pandas as pd 

import scipy.stats as stats
import statsmodels.api as sm

from sklearn import preprocessing 

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

import seaborn as sns
import csv
from pylab import *

import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
import string
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB


np.random.seed(0)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/wildalaska/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:

file = 'okcupidreduxdata.csv'
df = pd.read_csv(file)
df.head()

,Unnamed: 0,age,sex,about me,doingwithlife,strengths,whatothersnoticefirst,mediainterests,cantdowithout,oftenthinkabout,usualfriday,secret,messagemeif,think_revised,stopwords,thinknopunc,char_count,thinkredux,text_lemmatized,think_lemmatized
0,0,22,m,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,duality humorous things,1,duality humorous things,23,duality humorous things,"['duality', 'humorous', 'thing']","['duality', 'humorous', 'thing']"
1,5,29,m,"i'm an australian living in san francisco, but...",building awesome stuff. figuring out what's im...,imagining random shit. laughing at aforementio...,i have a big smile. i also get asked if i'm we...,"books: to kill a mockingbird, lord of the ring...","like everyone else, i love my friends and fami...",what my contribution to the world is going to ...,out with my friends!,i cried on my first day at school because a bi...,you're awesome.,contribution world going and/or be. what's bre...,11,contribution world going andor be whats breakf...,64,contribution world going andor be whats breakf...,"['contribution', 'world', 'going', 'andor', 'b...","['contribution', 'world', 'going', 'andor', 'b..."
2,9,37,m,my names jake. i'm a creative guy and i look f...,i have an apartment. i like to explore and che...,i'm good at finding creative solutions to prob...,i'm short,i like some tv. i love summer heights high and...,"music, my guitar contrast good food my bike my...",you should,send a message,and say hi.,you can rock the bells,NaN,2,NaN,0,NaN,[],[]
3,10,35,m,"update: i'm seeing someone, so off the market ...",i have three jobs. i've been doing sound and l...,"hugging, kissing, laughing, motivating people,...",my huge goofy smile,"i'm constantly reading, i read at what my frie...",family friends food women music reading,"snowboarding, food, women, goofy nerd stuff, a...",having dinner and drinks with friends and/or w...,i used to wish for a jetpack when blowing out ...,you are a complex woman with healthy self-este...,"snowboarding, food, women, goofy nerd stuff, a...",0,snowboarding food women goofy nerd stuff archi...,59,snowboarding food women goofy nerd stuff archi...,"['snowboarding', 'food', 'woman', 'goofy', 'ne...","['snowboarding', 'food', 'woman', 'goofy', 'ne..."
4,11,28,m,"i was born in wisconsin, grew up in iowa, and ...",i'm currently the youngest member on an intern...,i'm really good at a little bit of everything....,"the way i dress. some days it's hats, other da...",books = yes. avid reader. moves = eternal suns...,"guitar - even if i don't play it all the time,...",a little bit of everything. but mostly social ...,hanging out with a small group of friends--sta...,i'm picky when it comes to dating. i know what...,"if you know who you are, who you want, where y...",little bit everything. mostly social influence...,16,little bit everything mostly social influences...,170,little bit everything mostly social influences...,"['little', 'bit', 'everything', 'mostly', 'soc...","['little', 'bit', 'everything', 'mostly', 'soc..."


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29866 entries, 0 to 29865
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             29866 non-null  int64 
 1   age                    29866 non-null  int64 
 2   sex                    29866 non-null  object
 3   about me               29866 non-null  object
 4   doingwithlife          29866 non-null  object
 5   strengths              29866 non-null  object
 6   whatothersnoticefirst  29866 non-null  object
 7   mediainterests         29866 non-null  object
 8   cantdowithout          29866 non-null  object
 9   oftenthinkabout        29866 non-null  object
 10  usualfriday            29866 non-null  object
 11  secret                 29866 non-null  object
 12  messagemeif            29866 non-null  object
 13  think_revised          29819 non-null  object
 14  stopwords              29866 non-null  int64 
 15  thinknopunc        

maybe add in another column and look at similarities? or come up w/keywords for values ie family, fitness, and make a new list for people who have that value?

In [6]:
# testing out just one to see if this thing works

df1 = df[df['think_lemmatized'].str.contains("humor")]
print(df1['think_lemmatized'])

0                         ['duality', 'humorous', 'thing']
404      ['universe', 'a', 'booktv', 'show', 'universe'...
947      ['relationship', 'communication', 'biz', 'trav...
1346               ['humor', 'baseball', 'friend', 'work']
1909     ['architectural', 'portfolio', '75', 'time', '...
                               ...                        
28072    ['thinking', 'u', 'think', 'spamming', 'accoun...
28134    ['wonderful', 'life', 'is!', 'really', 'meetin...
28605    ['fortunate', 'create', 'reality', 'give', 'ge...
29063    ['humor', 'art', 'music', 'poetry', 'sex', 'lo...
29732    ['accent', 'absurdity', 'adventure', 'androgyn...
Name: think_lemmatized, Length: 87, dtype: object


In [7]:
df1.head()

,Unnamed: 0,age,sex,about me,doingwithlife,strengths,whatothersnoticefirst,mediainterests,cantdowithout,oftenthinkabout,usualfriday,secret,messagemeif,think_revised,stopwords,thinknopunc,char_count,thinkredux,text_lemmatized,think_lemmatized
0,0,22,m,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,duality humorous things,1,duality humorous things,23,duality humorous things,"['duality', 'humorous', 'thing']","['duality', 'humorous', 'thing']"
404,824,25,m,recently censored so i am harder to search by ...,"i work at ..., a company that makes inexpensiv...",i write comic strips that people seem to like....,- online: i can't spell - in person: i usually...,i will put a more comprehensive list up eventu...,stupidest question ... ever because i find thi...,"- i have a few ""universes"" (as in book/tv show...",at the hacker dojo,"- occasionally i go to bars, parties, ect., be...",in general i would greatly appreciate it if yo...,"""universes"" (as book/tv show universes) head e...",68,universes as booktv show universes head expand...,569,universes as booktv show universes head expand...,"['universe', 'a', 'booktv', 'show', 'universe'...","['universe', 'a', 'booktv', 'show', 'universe'..."
947,1908,49,m,"if you take any stock in the eneagram, i'm def...",i'm an entrepreneur who runs his own team buil...,"writing, solving crossword puzzles and sudokus...","my eyes or my smile, i suppose. it's hard to k...","when it comes to books and movies, i'm kind of...",in no particular order: movies books travel my...,"relationships, communication, my biz, travel, ...","watching a movie, or reading a great book at h...",that i cried at the latest muppet movie. what ...,...you agree that relationships are built on c...,"relationships, communication, biz, travel, pol...",53,relationships communication biz travel politic...,482,relationships communication biz travel politic...,"['relationship', 'communication', 'biz', 'trav...","['relationship', 'communication', 'biz', 'trav..."
1346,2805,34,m,"my interests include baseball, foreign/indepen...","taking one day at a time, loving the moment, b...",-making people laugh -search engine marketing ...,my boyish charm and my midwest hospitality(eve...,"y: the last man, the essential harlan ellison,...",1.time machines 2.dinosaurs 3.super intelligen...,-humor -baseball -friends -$$$ (work),if i'm not teaching at-risk juvenile delinquen...,"i read comic books, both independent or mainst...",you want to meet a someone with a sense of hum...,-humor -baseball -friends -$$$ (work),0,humor baseball friends work,28,humor baseball friends work,"['humor', 'baseball', 'friend', 'work']","['humor', 'baseball', 'friend', 'work']"
1909,3984,27,m,"""a detached, observant, and indifferent man. i...","what am i doing....well, i am in pursuit of ha...","/anything related to design: tattoo, drawing, ...","is that am different, indifferent, introverted...",books: fountainhead; atlas shrugged; the giver...,"ipod, laptop, iphone, brain, limbs w/ a majori...",my architectural portfolio (75% of the time); ...,everywhere and nowhere - the paradox of life.,".......eh, given that i am a secretive person,...","you feel like it. you are intelligent, inquis...",architectural portfolio (75% time); real meani...,70,architectural portfolio 75 time real meaning l...,946,architectural portfolio 75 time real meaning l...,"['architectural', 'portfolio', '75', 'time', '...","['architectural', 'portfolio', '75', 'time', '..."


In [5]:
df1.count()

Unnamed: 0               87
age                      87
sex                      87
about me                 87
doingwithlife            87
strengths                87
whatothersnoticefirst    87
mediainterests           87
cantdowithout            87
oftenthinkabout          87
usualfriday              87
secret                   87
messagemeif              87
think_revised            87
stopwords                87
thinknopunc              87
char_count               87
thinkredux               87
text_lemmatized          87
think_lemmatized         87
dtype: int64

Ok so it works - let's come up with some actual buzzwords for values, maybe combine w/bigrams?

make 3 dataframe lists 
find something to coolify it/combine it with - self awareness second column check?? or just use the info to come up 
w/better questions? see if those words also use other words - each list check for common bigrams***
make simple landing page w/questions and connect to a typeform? can use ccard 

list of buzzwords - 

value    -    buzzwords

fitness     shape, health, wellness, weight, train, gym, strength, yoga, body, nutrition, supplement, workout, exercise, martial, cardio, agile, baseball, basketball, wrestling, danc, movement, flexibility
personal growth       improve, learn, habit, skill, mind, how, new, try, alive, grow, train, way, why, bucket, better, fun, explore, work, business, experience
family/interpersonal      people, family, friends, relationship, communication, collaboration, competition, community, conversation, talk, hang, connect, weekend






In [22]:
df1 = df[df['think_lemmatized'].str.contains("shape") | df['think_lemmatized'].str.contains("health") 
         | df['think_lemmatized'].str.contains("wellness") | df['think_lemmatized'].str.contains("weight")
         | df['think_lemmatized'].str.contains("train") | df['think_lemmatized'].str.contains("gym")
         | df['think_lemmatized'].str.contains("strength") | df['think_lemmatized'].str.contains("yoga")
         | df['think_lemmatized'].str.contains("body") | df['think_lemmatized'].str.contains("nutrition")
         | df['think_lemmatized'].str.contains("supplement") | df['think_lemmatized'].str.contains("workout")
         | df['think_lemmatized'].str.contains("exercise") | df['think_lemmatized'].str.contains("martial")
         | df['think_lemmatized'].str.contains("cardio") | df['think_lemmatized'].str.contains("agile")
         | df['think_lemmatized'].str.contains("baseball") | df['think_lemmatized'].str.contains("basketball")
         | df['think_lemmatized'].str.contains("wrestl") | df['think_lemmatized'].str.contains("danc")
         | df['think_lemmatized'].str.contains("movement") | df['think_lemmatized'].str.contains("flex")
        
        
        ]

print(df1['think_lemmatized'])



4        ['little', 'bit', 'everything', 'mostly', 'soc...
9        ['build', 'empire', 'legacy', 'far', 'list', '...
13       ['symbiosis', 'go', 'weekend', 'politics', 'tr...
19       ['life', 'universe', 'everything', 'me', 'frie...
22       ['littlest', 'thing', 'catapulting', 'daydream...
                               ...                        
29778    ['life', 'going', 'cook', 'next', 'various', '...
29782                 ['want', 'day', 'ex', 'gym', 'work']
29806    ['doggy', 'society', 'politics', 'religion', '...
29824    ['anthropology', 'world', 'work', 'body', 'wor...
29858    ['bring', 'healthy', 'loving', 'world', 'film'...
Name: think_lemmatized, Length: 1975, dtype: object


In [24]:
df2 = df[df['think_lemmatized'].str.contains("improve") | df['think_lemmatized'].str.contains("learn") 
         | df['think_lemmatized'].str.contains("habit") | df['think_lemmatized'].str.contains("skill")
         | df['think_lemmatized'].str.contains("mind") | df['think_lemmatized'].str.contains("how")
         | df['think_lemmatized'].str.contains("new") | df['think_lemmatized'].str.contains("try")
         | df['think_lemmatized'].str.contains("alive") | df['think_lemmatized'].str.contains("grow")
         | df['think_lemmatized'].str.contains("train") | df['think_lemmatized'].str.contains("way")
         | df['think_lemmatized'].str.contains("why") | df['think_lemmatized'].str.contains("bucket")
         | df['think_lemmatized'].str.contains("better") | df['think_lemmatized'].str.contains("fun")
         | df['think_lemmatized'].str.contains("explore") | df['think_lemmatized'].str.contains("work")
         | df['think_lemmatized'].str.contains("business") | df['think_lemmatized'].str.contains("experience")     
        
        ]


print(df2['think_lemmatized'])


4        ['little', 'bit', 'everything', 'mostly', 'soc...
5        ['methodology', 'practicing', 'creative', 'ski...
7        ['like', 'thinking', 'way', 'improve', 'thing'...
8                   ['passion', 'searching', 'new', 'one']
12       ['leave', 'child', 'when', 'eventually', 'them...
                               ...                        
29855    ['how', 'better', 'parent', 'friend', 'daughte...
29858    ['bring', 'healthy', 'loving', 'world', 'film'...
29859    ['web', 'new', 'social', 'medium', 'collaborat...
29862    ['thinking', 'bus', 'tofrom', 'work', 'usually...
29863                 ['aside', 'work', 'improve', 'home']
Name: think_lemmatized, Length: 11922, dtype: object


In [25]:
df3 = df[df['think_lemmatized'].str.contains("people") | df['think_lemmatized'].str.contains("family") 
         | df['think_lemmatized'].str.contains("friends") | df['think_lemmatized'].str.contains("relationship")
         | df['think_lemmatized'].str.contains("communication") | df['think_lemmatized'].str.contains("collaboration")
         | df['think_lemmatized'].str.contains("competition") | df['think_lemmatized'].str.contains("community")
         | df['think_lemmatized'].str.contains("conversation") | df['think_lemmatized'].str.contains("talk")
         | df['think_lemmatized'].str.contains("hang") | df['think_lemmatized'].str.contains("connect")
         | df['think_lemmatized'].str.contains("weekend")    
        
        ]


print(df3['think_lemmatized'])

4        ['little', 'bit', 'everything', 'mostly', 'soc...
12       ['leave', 'child', 'when', 'eventually', 'them...
13       ['symbiosis', 'go', 'weekend', 'politics', 'tr...
20       ['build', 'thingsmake', 'thing', 'better', 'co...
23       ['im', 'going', 'travel', 'next', 'amazing', '...
                               ...                        
29849    ['scheming', 'love', 'coming', 'great', 'schem...
29853    ['everything', 'hidden', 'line', 'people', 'st...
29858    ['bring', 'healthy', 'loving', 'world', 'film'...
29859    ['web', 'new', 'social', 'medium', 'collaborat...
29865    ['sex', 'myself', 'people', 'amazing', 'everyt...
Name: think_lemmatized, Length: 7052, dtype: object


## check each list for common bigrams / mutual information scores 

In [57]:
post_str = ' '.join(df1['thinkredux'].tolist())

print (type(post_str))

<class 'str'>


In [58]:
#let's tokenize some stuff - bag of words

tokens = nltk.word_tokenize(post_str)
print(len(tokens))

72711


In [59]:
tokens_pos = nltk.pos_tag(tokens) #tagging pos in all the tokens plus the individual words themselves

In [44]:
from nltk.collocations import *


In [60]:
#list 1

bigram_measures1 = nltk.collocations.BigramAssocMeasures()

In [61]:
finder1 = BigramCollocationFinder.from_words(tokens_pos)


In [62]:
fitness_scored = finder1.score_ngrams(bigram_measures1.raw_freq)


In [63]:


fitness_scored[:50]

[((('lot', 'NN'), ('time', 'NN')), 0.0014303200341076315),
 ((('time', 'NN'), ('thinking', 'VBG')), 0.0013753077251034919),
 ((('im', 'NN'), ('going', 'VBG')), 0.0013065423388483174),
 ((('!', '.'), ('!', '.')), 0.0009764684848234793),
 ((('also', 'RB'), ('think', 'VBP')), 0.0006601477080496761),
 ((('spend', 'VBP'), ('lot', 'NN')), 0.0005776292445434666),
 ((('would', 'MD'), ('like', 'VB')), 0.0005776292445434666),
 ((('also', 'RB'), ('spend', 'VBP')), 0.000508863858288292),
 ((('health', 'NN'), ('care', 'NN')), 0.000508863858288292),
 ((('think', 'VBP'), ('lot', 'NN')), 0.0004951107810372571),
 ((('friends', 'NNS'), ('family', 'NN')), 0.00039883924028001264),
 ((('next', 'JJ'), ('trip', 'NN')), 0.00039883924028001264),
 ((('many', 'JJ'), ('people', 'NNS')), 0.0003850861630289777),
 ((('social', 'JJ'), ('justice', 'NN')), 0.0003850861630289777),
 ((('things', 'NNS'), ('like', 'IN')), 0.0003713330857779428),
 ((('martial', 'JJ'), ('arts', 'NNS')), 0.0003575800085269079),
 ((('family', 

In [64]:
#move on to info scores for d1

# mutual info scores

fitpmi_finder = BigramCollocationFinder.from_words(tokens_pos)

fitpmi_finder.apply_freq_filter(10)

In [65]:
fit_pmi_scored = fitpmi_finder.score_ngrams(bigram_measures1.pmi)


In [67]:
fit_pmi_scored [:50]

[((('hallelujah', 'NN'), ('hallelujah', 'NN')), 11.857104267437297),
 ((('gon', 'NN'), ('na', 'TO')), 11.14988601666514),
 ((('san', 'JJ'), ('francisco', 'JJ')), 10.962887501863886),
 ((('martial', 'JJ'), ('arts', 'NNS')), 10.925513231200453),
 ((('zombie', 'NN'), ('apocalypse', 'NN')), 10.827957921777779),
 ((('san', 'JJ'), ('francisco', 'NN')), 10.827957921777777),
 ((('video', 'NN'), ('games', 'NNS')), 10.49297367406497),
 ((('click', 'NN'), ('here', 'RB')), 10.018029056056346),
 ((('particular', 'JJ'), ('order', 'NN')), 9.895072117636316),
 ((('can', 'MD'), ('not', 'RB')), 9.32016328157908),
 ((('trying', 'VBG'), ('figure', 'NN')), 8.901958503221556),
 ((('right', 'RB'), ('now', 'RB')), 8.857564383863101),
 ((('cook', 'NN'), ('dinner', 'NN')), 8.740495080527438),
 ((('years', 'NNS'), ('ago', 'RB')), 8.62397375592179),
 ((('10', 'CD'), ('years', 'NNS')), 8.602440594372148),
 ((('social', 'JJ'), ('justice', 'NN')), 8.27424635504106),
 ((('human', 'JJ'), ('beings', 'NNS')), 8.25948354

In [68]:
#do everything for d2

post_str2 = ' '.join(df2['thinkredux'].tolist())

print (type(post_str2))


<class 'str'>


In [69]:
#let's tokenize some stuff - bag of words

tokens2 = nltk.word_tokenize(post_str2)
print(len(tokens2))

291327


In [70]:
tokens_pos2 = nltk.pos_tag(tokens2) #tagging pos in all the tokens plus the individual words themselves

In [71]:
bigram_measures2 = nltk.collocations.BigramAssocMeasures()

In [72]:
finder2 = BigramCollocationFinder.from_words(tokens_pos2)


In [73]:
pg_scored = finder2.score_ngrams(bigram_measures2.raw_freq)


In [74]:
pg_scored[:50]

[((('time', 'NN'), ('thinking', 'VBG')), 0.0022586303363574265),
 ((('lot', 'NN'), ('time', 'NN')), 0.002018350513340679),
 ((('im', 'NN'), ('going', 'VBG')), 0.0014382463692002457),
 ((('!', '.'), ('!', '.')), 0.001400488111297614),
 ((('world', 'NN'), ('better', 'RBR')), 0.000892467914062205),
 ((('also', 'RB'), ('think', 'VBP')), 0.0008581422250598125),
 ((('better', 'RBR'), ('place', 'NN')), 0.0007380023135514388),
 ((('would', 'MD'), ('like', 'VB')), 0.0006693509355466538),
 ((('make', 'VBP'), ('world', 'NN')), 0.0006521880910454575),
 ((('spend', 'VB'), ('lot', 'NN')), 0.0006521880910454575),
 ((('spend', 'VBP'), ('lot', 'NN')), 0.0006281601087437828),
 ((('things', 'NNS'), ('work', 'VBP')), 0.0005595087307389978),
 ((('think', 'VBP'), ('lot', 'NN')), 0.0005595087307389978),
 ((('friends', 'NNS'), ('family', 'NN')), 0.000535480748437323),
 ((('better', 'RBR'), ('person', 'NN')), 0.0005011550594349305),
 ((('also', 'RB'), ('spend', 'VBP')), 0.0004702619393327773),
 ((('right', 'RB

In [75]:
# mutual info scores

pgpmi_finder = BigramCollocationFinder.from_words(tokens_pos2)

pgpmi_finder.apply_freq_filter(10)

In [76]:
pg_pmi_scored = pgpmi_finder.score_ngrams(bigram_measures2.pmi)


In [78]:
pg_pmi_scored [:50]

[((('hallelujah', 'NN'), ('hallelujah', 'NN')), 13.859498140937106),
 ((('golden', 'JJ'), ('gate', 'NN')), 13.508423700390226),
 ((('ice', 'NN'), ('cream', 'NN')), 12.854129178560713),
 ((('blah', 'NN'), ('blah', 'NN')), 12.830351795277588),
 ((('martial', 'JJ'), ('arts', 'NNS')), 12.489314877442522),
 ((('artificial', 'JJ'), ('intelligence', 'NN')), 12.04375543338678),
 ((('global', 'JJ'), ('warming', 'VBG')), 11.684131054426546),
 ((('ok', 'JJ'), ('cupid', 'JJ')), 11.660426793835274),
 ((('black', 'JJ'), ('holes', 'NNS')), 11.556429073301057),
 ((('san', 'NN'), ('francisco', 'NN')), 11.409465220302057),
 ((('san', 'JJ'), ('francisco', 'NN')), 11.164981210840041),
 ((('quite', 'RB'), ('bit', 'JJ')), 11.096997454663763),
 ((('zombie', 'NN'), ('apocalypse', 'NN')), 11.022996873219984),
 ((('san', 'JJ'), ('francisco', 'JJ')), 11.020422929556158),
 ((('hot', 'JJ'), ('springs', 'NNS')), 11.009940720394537),
 ((('gon', 'NN'), ('na', 'TO')), 10.933111369702793),
 ((('gon', 'VBG'), ('na', 'TO

In [79]:
#do this again for list 3 - so far the pmi scores have differed pretty well
post_str3 = ' '.join(df3['thinkredux'].tolist())

print (type(post_str3))


<class 'str'>


In [80]:
#let's tokenize some stuff - bag of words

tokens3 = nltk.word_tokenize(post_str3)
print(len(tokens3))

191503


In [81]:
tokens_pos3 = nltk.pos_tag(tokens3) #tagging pos in all the tokens plus the individual words themselves

In [82]:
bigram_measures3 = nltk.collocations.BigramAssocMeasures()

In [83]:
finder3 = BigramCollocationFinder.from_words(tokens_pos3)


In [84]:
people_scored = finder3.score_ngrams(bigram_measures3.raw_freq)


In [85]:
people_scored [:50]

[((('time', 'NN'), ('thinking', 'VBG')), 0.0019581938664146254),
 ((('lot', 'NN'), ('time', 'NN')), 0.0016918795005822363),
 ((('friends', 'NNS'), ('family', 'NN')), 0.0014621180869229203),
 ((('im', 'NN'), ('going', 'VBG')), 0.0013472373800932622),
 ((('!', '.'), ('!', '.')), 0.0011070322658130682),
 ((('family', 'NN'), ('friends', 'VBZ')), 0.0009555986068103372),
 ((('also', 'RB'), ('think', 'VBP')), 0.0007884993968762891),
 ((('many', 'JJ'), ('people', 'NNS')), 0.0007153934925301431),
 ((('family', 'NN'), ('friends', 'NNS')), 0.0006579531391153141),
 ((('would', 'MD'), ('like', 'VB')), 0.0006422875881839971),
 ((('people', 'NNS'), ('think', 'VBP')), 0.0006214001869422411),
 ((('people', 'NNS'), ('love', 'VBP')), 0.0006109564863213631),
 ((('people', 'NNS'), ('life', 'NN')), 0.0006005127857004851),
 ((('spend', 'VBP'), ('lot', 'NN')), 0.0005744035341482901),
 ((('spend', 'VB'), ('lot', 'NN')), 0.0005639598335274121),
 ((('think', 'VBP'), ('lot', 'NN')), 0.0005274068813543391),
 ((('m

In [86]:
# mutual info scores

peoplepmi_finder = BigramCollocationFinder.from_words(tokens_pos3)

peoplepmi_finder.apply_freq_filter(10)

In [87]:
peoplepmi_scored = peoplepmi_finder.score_ngrams(bigram_measures3.pmi)


In [88]:
peoplepmi_scored [:50]

[((('ted', 'VBD'), ('talks', 'NNS')), 13.409503943626781),
 ((('hallelujah', 'NN'), ('hallelujah', 'NN')), 13.25422571814887),
 ((('ice', 'NN'), ('cream', 'NN')), 12.771906744323738),
 ((('zombie', 'NN'), ('apocalypse', 'NN')), 11.177326241040983),
 ((('san', 'JJ'), ('francisco', 'JJ')), 11.140884245072924),
 ((('bay', 'VBP'), ('area', 'NN')), 11.132994973498631),
 ((('bay', 'JJ'), ('area', 'NN')), 11.093895153809695),
 ((('san', 'JJ'), ('francisco', 'NN')), 11.074962852127216),
 ((('gon', 'NN'), ('na', 'TO')), 11.001419204989698),
 ((('self', 'PRP'), ('improvement', 'NN')), 10.89165563876416),
 ((('quantum', 'NN'), ('physics', 'NNS')), 10.643728125320576),
 ((('tv', 'NN'), ('shows', 'NNS')), 10.588586571128111),
 ((('fair', 'JJ'), ('amount', 'NN')), 10.585075508210233),
 ((('online', 'NN'), ('dating', 'VBG')), 10.49607050215149),
 ((('got', 'VBD'), ('ta', 'JJ')), 10.458787107053633),
 ((('string', 'VBG'), ('theory', 'NN')), 10.411554683191541),
 ((('bay', 'NN'), ('area', 'NN')), 10.36

seeing both similarities and differencees btw the 3 priority groups:

## summary of differences


## summary of differences

# fit

present moment

social movements

body language


# pg

golden gate

blah blah

artificial intelligence

black holes

quite bit

hot springs

pursuit happiness

daily basis

real estate

renewable energy

bucket list

rock climbing

problem solving


# people

ted talks as the top one

fair amount 

road trips

grow up

personal growth

current events

positive impact

care about

someone elses

etc etc

big picture

travel destination

wish could

list goes

lucky am

new york


# Patterns of distinction

personal growth -  there’s a productivity/goal based theme
people - focus on others, philosophy, big picture life stuff
fitness - focused on small actions that can add up

Let's try to tease out more of their largest core value / reason for doing snowsports or in life in general, ask questions that harder to lie about / easier to answer intuitively for most people

# Question ideas:

original question in dataset - what do you often think about? 

What is this trying to get at, and what can we use our patterns of distinction to tease out? - problem solving process, subconscious, priorities, values, philosophy, what they're curious about

- what would you make better about yourself instantly if you could, even if you’re already good at it?
- when was the last time you found yourself wishing for more time?
- philosophy question - how would you get your dog to bring you a slice of pizza?
- what’s one of your favorite topics and how would you teach someone the most basic thing on it?
- what was the last dream you remember having when sleeping?
- how do you approach planning / running your vacations? 
- what was your day yesterday like?
- tell us about someone you really dislike
- last time you felt truly alive / wanted time to stop to savor more
- what’s something that took you a long or short time to learn?

- what is a metric of success/fulfillment for you personally in your life?
- tell us about your most recent good / bad day
- what are you looking forward to next week?

Top 6 Q's:
- what are you looking forward to next week?
- how do you like to plan your vacations / spend your time during them? 
- tell us about someone you truly dislike and why?
- philosophy question - how would you get your dog to bring you a slice of pizza?
- last time you felt truly alive / wanted time to stop to savor more / or where would you go back in time in your life to experience something twice?
- what new skill do you most want to learn right now / or what current skill do you most want to improve? and when was the last time you worked on this skill?


